In [ ]:
from traitlets import Unicode, Bool, validate, TraitError, observe
from ipywidgets import DOMWidget, register
import time
import threading

In [ ]:
%%javascript

require.undef('email_widget');

define('email_widget', ["@jupyter-widgets/base"], function(widgets) {
    var EmailView = widgets.DOMWidgetView.extend({

        render: function() {
            // Key concepts:
            // - this.model: the model associated with a view instance
            // - this.el   : DOM element associated with the view
            this.email_input = document.createElement('input');
            this.email_input.type = 'email';
            this.email_input.value = this.model.get('pipe_p2j');
            this.email_input.disabled = this.model.get('disabled');
            this.el.appendChild(this.email_input);
            
            // Python -> JavaScript update
            this.model.on('change:pipe_p2j', this.on_pipe_p2j, this);
            this.model.on('change:disabled', this.disabled_changed, this);
            
            // JavaScript -> Python update
            this.email_input.onchange = this.input_changed.bind(this);
        },
        
        on_pipe_p2j: function() {
            this.email_input.value = this.model.get('pipe_p2j');
        },
        
        disabled_changed: function() {
            this.email_input.disabled = this.model.get('disabled');
        },
        
        input_changed: function() {
            this.model.set('pipe_p2j', this.email_input.value);
            // Use touch(), recommended by
            // https://github.com/jupyter-widgets/ipywidgets/issues/1783#issuecomment-340312257
            this.touch()
            // this.model.save_changes();
        },
    });
    
    return {
        EmailView: EmailView
    }
});

In [ ]:
def change_value_five_times(widget):
    def func():
        for i in range(5):
            time.sleep(1)
            widget.pipe_p2j = f"update_{i}@gmail.com"
    return func

@register
class Email(DOMWidget):
    _view_name = Unicode('EmailView').tag(sync=True)
    _view_module = Unicode('email_widget').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    
    # Attributes
    pipe_p2j = Unicode('example@example.com', help="Python to JS pipe.").tag(sync=True)
    disabled = Bool(False, help="Enable or disable user changes.").tag(sync=True)

    @observe('pipe_p2j')
    def _observe_pipe_p2j(self, change):
        print(f"_observe_value: {change['old']}")
        print(f"_observe_value: {change['new']}")

In [ ]:
email = Email(pipe_p2j='john.doe@domain.com', disabled=False)
email

In [ ]:
thread = threading.Thread(target=change_value_five_times(email))
thread.start()

In [ ]:
# Python to JS update
email.value = 'gwen@domain.com'

In [ ]:
# Now, in the Widget, change value manually
email.value